# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 13 2023 8:30AM,259130,10,WT5633 Out,Methapharm-G,Released
1,Mar 10 2023 3:46PM,259128,15,30014586,"Alliance Pharma, Inc.",Released
2,Mar 10 2023 3:46PM,259128,15,9183162-T,"Brookfield Pharmaceuticals, LLC",Released
3,Mar 10 2023 2:58PM,259121,10,MSP219423,Methapharm-G,Released
4,Mar 10 2023 2:53PM,259119,10,MSP218388-BO,Methapharm-G,Released
5,Mar 10 2023 2:46PM,259117,10,MSP218278-BO,Methapharm-G,Released
6,Mar 10 2023 2:45PM,259116,10,MSP218915-BO,Methapharm-G,Released
7,Mar 10 2023 2:37PM,259114,10,MSP218818-BO,Methapharm-G,Released
8,Mar 10 2023 2:34PM,259113,10,MSP218703-BO,Methapharm-G,Released
9,Mar 10 2023 2:32PM,259112,10,MSP218534-BO,Methapharm-G,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,259117,Released,1
20,259119,Released,1
21,259121,Released,1
22,259128,Released,2
23,259130,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259117,NaN,NaN,1.0
259119,NaN,NaN,1.0
259121,NaN,NaN,1.0
259128,NaN,NaN,2.0
259130,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259057,21.0,12.0,0.0
259071,0.0,1.0,0.0
259084,0.0,0.0,1.0
259091,0.0,1.0,0.0
259092,0.0,0.0,16.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259057,21,12,0
259071,0,1,0
259084,0,0,1
259091,0,1,0
259092,0,0,16


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259057,21,12,0
1,259071,0,1,0
2,259084,0,0,1
3,259091,0,1,0
4,259092,0,0,16


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259057,21,12,
1,259071,,1,
2,259084,,,1
3,259091,,1,
4,259092,,,16


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 13 2023 8:30AM,259130,10,Methapharm-G
1,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc."
2,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC"
3,Mar 10 2023 2:58PM,259121,10,Methapharm-G
4,Mar 10 2023 2:53PM,259119,10,Methapharm-G
5,Mar 10 2023 2:46PM,259117,10,Methapharm-G
6,Mar 10 2023 2:45PM,259116,10,Methapharm-G
7,Mar 10 2023 2:37PM,259114,10,Methapharm-G
8,Mar 10 2023 2:34PM,259113,10,Methapharm-G
9,Mar 10 2023 2:32PM,259112,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 13 2023 8:30AM,259130,10,Methapharm-G,,,1
1,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",,,2
2,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",,,2
3,Mar 10 2023 2:58PM,259121,10,Methapharm-G,,,1
4,Mar 10 2023 2:53PM,259119,10,Methapharm-G,,,1
5,Mar 10 2023 2:46PM,259117,10,Methapharm-G,,,1
6,Mar 10 2023 2:45PM,259116,10,Methapharm-G,,,1
7,Mar 10 2023 2:37PM,259114,10,Methapharm-G,,,1
8,Mar 10 2023 2:34PM,259113,10,Methapharm-G,,,1
9,Mar 10 2023 2:32PM,259112,10,Methapharm-G,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 8:30AM,259130,10,Methapharm-G,1,,
1,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",2,,
2,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",2,,
3,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1,,
4,Mar 10 2023 2:53PM,259119,10,Methapharm-G,1,,
5,Mar 10 2023 2:46PM,259117,10,Methapharm-G,1,,
6,Mar 10 2023 2:45PM,259116,10,Methapharm-G,1,,
7,Mar 10 2023 2:37PM,259114,10,Methapharm-G,1,,
8,Mar 10 2023 2:34PM,259113,10,Methapharm-G,1,,
9,Mar 10 2023 2:32PM,259112,10,Methapharm-G,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 8:30AM,259130,10,Methapharm-G,1,,
1,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",2,,
2,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",2,,
3,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1,,
4,Mar 10 2023 2:53PM,259119,10,Methapharm-G,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 8:30AM,259130,10,Methapharm-G,1.0,NaN,NaN
1,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",2.0,NaN,NaN
2,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",2.0,NaN,NaN
3,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1.0,NaN,NaN
4,Mar 10 2023 2:53PM,259119,10,Methapharm-G,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 8:30AM,259130,10,Methapharm-G,1.0,0.0,0.0
1,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",2.0,0.0,0.0
2,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",2.0,0.0,0.0
3,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1.0,0.0,0.0
4,Mar 10 2023 2:53PM,259119,10,Methapharm-G,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3627567,30.0,0.0,0.0
12,259091,0.0,1.0,0.0
15,518256,4.0,0.0,0.0
18,259084,1.0,0.0,0.0
19,777252,3.0,16.0,21.0
21,259071,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3627567,30.0,0.0,0.0
1,12,259091,0.0,1.0,0.0
2,15,518256,4.0,0.0,0.0
3,18,259084,1.0,0.0,0.0
4,19,777252,3.0,16.0,21.0
5,21,259071,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,30.0,0.0,0.0
1,12,0.0,1.0,0.0
2,15,4.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,3.0,16.0,21.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,30.0
1,12,Released,0.0
2,15,Released,4.0
3,18,Released,1.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,21.0,0.0
Executing,0.0,1.0,0.0,0.0,16.0,1.0
Released,30.0,0.0,4.0,1.0,3.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,21
0,Completed,0.0,0.0,0.0,0.0,21.0,0.0
1,Executing,0.0,1.0,0.0,0.0,16.0,1.0
2,Released,30.0,0.0,4.0,1.0,3.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,21
0,Completed,0.0,0.0,0.0,0.0,21.0,0.0
1,Executing,0.0,1.0,0.0,0.0,16.0,1.0
2,Released,30.0,0.0,4.0,1.0,3.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()